In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
#Declare url variables
base_url = 'https://footballdatabase.com'
url = base_url + '/clubs-scores/atletico-madrid/1'

#Request the desired url (FootballDataBase)
page = requests.get(url)
#Parser the requested page
soups = []
soups.append(BeautifulSoup(page.content, "html.parser"))

In [4]:
#Get every link from the navigation buttons panel
links_bar = soups[0].find("ul", class_="pagination pagination-sm")
link_buttons = links_bar.find_all('li')

#Create a list with every url termination
links = [link.find('a')['href'] for link in link_buttons]

In [5]:
#Loop through every url termination (Starts at 1 because the first was already appended)
for link in links[1:]:
    #Determine the new url
    url = base_url + link
    #Requests the current url
    page = requests.get(url)
    #Append to the previoulsy requested pages
    soups.append(BeautifulSoup(page.content, "html.parser"))

In [6]:
#Get every Results Table from every page
results_tables = []
for soup in soups:
    for table in soup.find_all("div", class_ = "col-md-8"):
        if table.get("class") == ["col-md-8"]:
            results_tables.append(table)

In [7]:
#Get the columns titles from the first page to create the DataFrame
first_row = results_tables[0].find('div')
column_titles = [data.get_text() for data in first_row.find_all('div')]
#Create the DataFrame
df = pd.DataFrame(columns = column_titles)

In [8]:
df

,Competition,Date,Home Team,Score,Away Team


In [9]:
data_index = 0
for result_table in results_tables:
    results = result_table.find_all('div', class_ = "club-gamelist-match")
    for result in results[1:]:
        result_data = [data.get_text() for data in result.find_all('a', class_="club-gamelist-match-info")]
        for data in result.find_all('div'):
            result_data.append(data.get_text())
        df.loc[data_index] = result_data
        data_index += 1

In [10]:
df

,Competition,Date,Home Team,Score,Away Team
0,Spain Liga BBVA 2025/2026,01 November 2025,Atlético Madrid,3 - 0,Sevilla
1,Spain Liga BBVA 2025/2026,27 October 2025,Real Betis,0 - 2,Atlético Madrid
2,UEFA Champions League 2025/2026,21 October 2025,Arsenal,4 - 0,Atlético Madrid
3,Spain Liga BBVA 2025/2026,18 October 2025,Atlético Madrid,1 - 0,Osasuna
4,Spain Liga BBVA 2025/2026,05 October 2025,Celta Vigo,1 - 1,Atlético Madrid
...,...,...,...,...,...
793,UEFA Champions League 2009/10 (Group D),15 September 2009,Atlético Madrid,0 - 0,APOEL Nicosia
794,Spain Liga BBVA 2009/10,12 September 2009,Atlético Madrid,1 - 1,Racing Santander
795,Spain Liga BBVA 2009/10,30 August 2009,Málaga,3 - 0,Atlético Madrid
796,UEFA Champions League 2009/10 (Qualifying Match),25 August 2009,Atlético Madrid,2 - 0,Panathinaikos


In [11]:
#Split Score column to HomeScore and AwayScore
df[["Home Score_Str", "Away Score_Str"]] = df["Score"].str.split('-', expand = True)
#Set Home Score_Str and Away Score_Str to Ints
df["Home Score"] = df["Home Score_Str"].astype(int)
df["Away Score"] = df["Away Score_Str"].astype(int)
#Drop Score, Home Score_Str and Away Score_Str columns
df.drop(columns = ["Score", "Home Score_Str", "Away Score_Str"], inplace = True)

In [12]:
#Get the Season Column
#Create a pattern to find the season (4 chars before '/' and 2 or 4 chars after)
pattern = r'(\d{4}/\d{2,4})'
#Extract those values into a new column
df["Season"] = df["Competition"].str.extract(pattern)

In [13]:
#Now Season values are "XXXX/XXXX" or "XXXX/XX"
#I am going to unify every value to become "XXXX/XX"
df.loc[df['Season'].str.len() > 7, 'Season'] = df['Season'].str[:5] + df['Season'].str[-2:]

In [14]:
#Standarize Competitions to be the same name regardless the Season
#Define conditions
conditions = [
    df["Competition"].str.contains("Spain Liga BBVA", na=False),
    df["Competition"].str.contains("UEFA Champions League", na=False),
    df["Competition"].str.contains("UEFA Europa League", na=False),
    df["Competition"].str.contains("FIFA Club World Cup", na=False),
    df["Competition"].str.contains("UEFA Super Cup", na=False)
]
#Define the names I want for the competitions
choices = [
    "Spain Liga",
    "UEFA Champions League",
    "UEFA Europa League",
    "FIFA Club World Cup",
    "UEFA Super Cup"
]
#Make the change
df["Competition"] = np.select(conditions, choices, default = "Other")

In [15]:
df

,Competition,Date,Home Team,Away Team,Home Score,Away Score,Season
0,Spain Liga,01 November 2025,Atlético Madrid,Sevilla,3,0,2025/26
1,Spain Liga,27 October 2025,Real Betis,Atlético Madrid,0,2,2025/26
2,UEFA Champions League,21 October 2025,Arsenal,Atlético Madrid,4,0,2025/26
3,Spain Liga,18 October 2025,Atlético Madrid,Osasuna,1,0,2025/26
4,Spain Liga,05 October 2025,Celta Vigo,Atlético Madrid,1,1,2025/26
...,...,...,...,...,...,...,...
793,UEFA Champions League,15 September 2009,Atlético Madrid,APOEL Nicosia,0,0,2009/10
794,Spain Liga,12 September 2009,Atlético Madrid,Racing Santander,1,1,2009/10
795,Spain Liga,30 August 2009,Málaga,Atlético Madrid,3,0,2009/10
796,UEFA Champions League,25 August 2009,Atlético Madrid,Panathinaikos,2,0,2009/10


In [16]:
#Create new Collumn to indicate if Atletico was playing at Home or Away
df["Place"] = np.where(df["Home Team"] == "Atlético Madrid", "Home", "Away")

In [17]:
#Set the Date column to date format (dd MM YYYY)
df["Date"] = pd.to_datetime(df["Date"], format = '%d %B %Y')
#Create new column with the Year
df["Year"] = df["Date"].dt.year
#Create new column with the Month
df["Month"] = df["Date"].dt.month

In [26]:
df

,Competition,Date,Home Team,Away Team,Home Score,Away Score,Season,Place,Year,Month,Atletico Goals,Opponent Goals,Goal Difference,Match Status
0,Spain Liga,2025-11-01,Atletico Madrid,Sevilla,3,0,2025/26,Home,2025,11,3,0,3,Win
1,Spain Liga,2025-10-27,Real Betis,Atletico Madrid,0,2,2025/26,Away,2025,10,2,0,2,Win
2,UEFA Champions League,2025-10-21,Arsenal,Atletico Madrid,4,0,2025/26,Away,2025,10,0,4,-4,Loss
3,Spain Liga,2025-10-18,Atletico Madrid,Osasuna,1,0,2025/26,Home,2025,10,1,0,1,Win
4,Spain Liga,2025-10-05,Celta Vigo,Atletico Madrid,1,1,2025/26,Away,2025,10,1,1,0,Draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,UEFA Champions League,2009-09-15,Atletico Madrid,APOEL Nicosia,0,0,2009/10,Home,2009,9,0,0,0,Draw
778,Spain Liga,2009-09-12,Atletico Madrid,Racing Santander,1,1,2009/10,Home,2009,9,1,1,0,Draw
779,Spain Liga,2009-08-30,Malaga,Atletico Madrid,3,0,2009/10,Away,2009,8,0,3,-3,Loss
780,UEFA Champions League,2009-08-25,Atletico Madrid,Panathinaikos,2,0,2009/10,Home,2009,8,2,0,2,Win


In [19]:
#Remove every row with null values
df.fillna('', inplace = True)
for x in df.index:
    if df.loc[x, "Season"] == '':
        df.drop(x, inplace = True)

In [20]:
#Restart the index value because some rows were eliminated
df.reset_index(drop = True, inplace = True)

In [21]:
#Get Atleti Goals per match
df["Atletico Goals"] = np.where(df["Home Team"] == "Atlético Madrid", df["Home Score"], df["Away Score"])
#Get Opponent Goals per match
df["Opponent Goals"] = np.where(df["Home Team"] == "Atlético Madrid", df["Away Score"], df["Home Score"])
#Get Gaols Difference
df["Goal Difference"] = df["Atletico Goals"] - df["Opponent Goals"]

In [23]:
#Create a column for the Match Status (Win, Loss, Draw)
conditions = [
    df["Goal Difference"] > 0,
    df["Goal Difference"] < 0
]

choices = [
    "Win",
    "Loss"
]

#Create columns
df["Match Status"] = np.select(conditions, choices, default = "Draw")

In [25]:
#Remove the accent from every team name (Home and Away)
#Define a dictionary with the character to find and its replacement
replacements = {
    'á': 'a', 'Á': 'A',
    'é': 'e', 'É': 'E',
    'í': 'i', 'Í': 'I',
    'ó': 'o', 'Ó': 'O',
    'ú': 'u', 'Ú': 'U',
    'ñ': 'n', 'Ñ': 'N',
}

#Loop only the desired columns
for col in ['Home Team', 'Away Team']:
    # regex = True to replace more than one character if there are more
    for accent, standard in replacements.items():
        df[col] = df[col].str.replace(accent, standard, regex=False)

In [27]:
#Export it as a CSV
df.to_csv(r'D:\DataAnalystBootcamp\Atleti\Results.csv')